---

# Allo allo, la terre appelle la lune!
### GoPiGo 101 - Série d'exercices 3
##### Manipulation de la **télécommande**. 

Pour cet exercice, vous devez avoir connecté le récepteur infrarouge à un port analogique/numérique (`AD1` ou `AD2`). [Voir cette image](https://gopigo3.readthedocs.io/en/master/api-basic/structure.html#hardware-ports).

---

Les fonctions suivantes permettent la manipulation de la télécommande :
 - `EasyGoPiGo3.init_remote` : crée un objet `easysensors.Remote` permettant d'interagir avec la télécommande
 - `Remote.read` : retourne un code correspondant à la touche lue (un **entier** correspondant à l'index d'oû se trouve la touche dans ce tableau)
   - `keycodes = ['', 'up', 'left', 'ok', 'right', 'down', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '0', '#']`
   - la valeur `keycodes[0]` (la valeur `''`) indique qu'il n'y a aucune touche d'appuyée
   - une lecture erronnée retourne -1
 - `Remote.get_remote_code` : retourne une **chaîne de caractères** correspondant à la touche lue (soit la chaîne de caractères spécifiée par le tableau `keycodes`)


**Important** : Vous remarquerez que la performance associée à l'usage de la télécommande est plutôt faible. D'ailleurs, on peut observer :
 - le temps de réponse est relativement lent
 - un seul bouton à la fois peut être détecté : si plusieurs touches sont actives, alors le résultat est équivalent à aucune touche
 - la réception est sensible à la distance et aux obstacles intermédiaires
 - la qualité de la réception est intermittente et il est relativement difficile de maintenir toujours la même performance
 - il n'y a pas de mécanisme de gestion d'action comme des écouteurs de style `on_key_pressed` et `on_key_released`
 - dans certains cas, la fonction `get_remote_code` ne fonctionne pas bien alors que la fonction `read` semble toujours foncionner

Il faudra en tenir compte de toutes ces caractéristiques dans vos programmes.

### Démonstration

In [2]:
import easygopigo3 as gpg
import time

robot = gpg.EasyGoPiGo3()
remote_control_port = 'AD1'
remote_control = robot.init_remote(port=remote_control_port)


# débute l'acquisition tant que la touche 'ok' n'a pas été appuyée
print('Acquisition de la télécommande en cours...')
key = ''
longuest_string = 20 # longueur arbitraire, ne sert que pour l'affichage
while key != 'ok':
    key = remote_control.get_remote_code()
    print(f'\r{key if key != "" else "   ~~~[no key]~~~"}', sep='', end=' ' * longuest_string)
    
print('\n','...acquisition terminée!')


del remote_control
del remote_control_port
del robot
del time
del gpg

ModuleNotFoundError: No module named 'easygopigo3'

---
### Préparation
Faites la mise en place du code commun pour cette série d'exercices

In [ ]:
# Mise en place du code commun
import easygopigo3 as gpg
import time

robot = gpg.EasyGoPiGo3()
remote_control_port = 'AD1'
remote_control = robot.init_remote(port=remote_control_port)

keycodes = ['', 'up', 'left', 'ok', 'right', 'down', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '0', '#']

---
### Exercice 3.1. 
Selon les instructions données par la télécommande, faites les opérations suivantes sur le déplacement du robot :
- `up` avance
- `down` recule
- `right` tourne à droite
- `left` tourne à gauche
- `ok` arrête
- `0` terminer le programme

In [ ]:
# Solution
longuest_string = 20
key = 99
while key != 3:
    key = remote_control.read()
    print(f'\r{key if key != "" else "   ~~~[no key]~~~"}', sep='', end=' ' * longuest_string)
    if(key == 1):
        robot.forward()
    if(key == 2):
        robot.left()
        
# Solution
print('Contrôle manuel du robot en cours...')
key = ''
while key != '0':
    key = remote_control.get_remote_code()
    if key == 'ok':
        robot.stop()
    elif key == 'up':
        robot.forward()
    elif key == 'down':
        robot.backward()
    elif key == 'right':
        robot.right()
    elif key == 'left':
        robot.left()
        
print('... fin du contrôle manuel!')

---
### Exercice 3.2.
Utilisez le robot pour faire une mini calculatrice! 

Ces touches de la télécommande servent à :
 - de `0` à `9` : le nombre correspondant
 - `left` : l'addition
 - `right` : la soustraction
 - `ok` : effectue le calcul des informations entrées et termine le programme.
 
_Indice_ simplifiant cet exercice : la fonction `eval` permet l'exécution d'une instruction `Python` existant sous forme de chaîne de caractères. Évidemment, mettre cette _évaluation_ sous surveillance `try` permet de gérer les cas où la chaîne de caractères est invalide.

Par exemple, `eval('123 + 321')` donne le résultat `444`.

In [ ]:
# Solution 
key = ''
buffer = ''
operation = ''
print('Acquisition de la télécommande en cours...')
while key != 3:
    key = remote_control.read()
    if key != buffer :
        if key == 2:
            operation = operation + "+"
        elif key == 4:
            operation = operation + "-"
        elif key != 0 and key != 3 :
            operation = operation + keycodes[key]
        buffer = key
        
        
print(eval(operation))
        
# Solution
print('Mode calculatrice', 'Veuillez saisir une équation valide...', sep='\n')

key = ''
equation = ''

while key != 'ok':
    key = remote_control.get_remote_code()
    if key >= '0' and key <= '9':
        equation = equation + key
    elif key == 'left':
        equation = equation + ' + '
    elif key == 'right':
        equation = equation + ' - '
    print(f'\r{equation}', sep='', end=' ' * (len(equation) + 1))
    time.sleep(0.25) # nombre arbitraire mais difficile à calibrer

erase = ' ' * (len(equation) + 1)
try:
    solution = eval(equation)
    text = f'\r{equation} = {solution}'
except:
    text = '--- équation invalide ---'
finally:    
    print(f'{text}{erase}')
    print('Fin du mode calculatrice.') 
   

---
### Exercice 3.3.
Faites une boucle de _x_ secondes qui tourne le plus rapidement possible. Cette boucle doit calculer le nombre de fois que chacune des 17 touches est appuyés et l'afficher à l'écran. Vous devez utiliser la fonction `Remote.read`. Ne faites pas de `wait` pour ralentir le programme.

Faites une version qui affiche le résultat après le temps écoulé et une autre version qui affiche le résultat pendant les mesures. Dans tous les cas, faites afficher le nombre total de touches appuyées à la toute fin.

In [ ]:
# Solution
# ...
duree = 25

temp = {}

for i in keycodes:
    temp[i] = 0


while duree > 0:
    # insert code
    key = remote_control.read()
    
    
    temp[keycodes[key]] = temp.get(keycodes[key]) + 1
    
    
    print(temp)
    duree -= 1
    time.sleep(1)

# Solution
ref_time = time.perf_counter()
duration = 10.0 # en secondes
display_result_in_test = True # changer cette variable d'état pour activer ou désactiver l'affichage pendant l'acquisition
max_view_len = 0

keycodes = ['~', 'up', 'left', 'ok', 'right', 'down', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '0', '#']
cumul = [0 for _ in range(len(keycodes))]

print('Calcul du nombre de fois où chaque touche de la télécommande est lue...')
while time.perf_counter() - ref_time <= duration:
    key_code = remote_control.read()
    cumul[key_code] += 1
    
    if display_result_in_test:
        view = ' - '.join([f'{key}[{count}]' for key, count in zip(keycodes, cumul)])
        view_len = len(view)
        if view_len > max_view_len:
            max_view_len = view_len
        print(f'\r{view}', end=' ' * max_view_len)

if not display_result_in_test:
    print(' - '.join([f'{key}[{count}]' for key, count in zip(keycodes, cumul)]))
    
print(f'Au total, {sum(cumul)} acquisitions ont été faites.')
    
print('... fin!')

---
### Exercice 3.4.
Utilisez la télécommande pour contrôler les phares. Voici le actions à donner aux touches :
 - `1`, `4` et `7` : faites clignoter uniquement le phare gauche aux fréquences suivantes : 1.5 secondes, 1.0 secondes et 0.5 secondes
 - `2`, `5` et `8` : faites clignoter les 2 phares aux fréquences suivantes : 1.5 secondes, 1.0 secondes et 0.5 secondes
 - `3`, `6` et `9` : faites clignoter uniquement le phare droit aux fréquences suivantes : 1.5 secondes, 1.0 secondes et 0.5 secondes
 - `0` : arrête tout clignotement
 - `ok` arrête le programme

Assurez-vous que les phares soient éteints à la sortie du programme.

**Attention** : vous ne pouvez pas faire de `wait` pour bloquer le programme!

In [ ]:
# Solution
print('Opération manuelle des phares...')

blink_left = False
blink_right = False
current_set_on = True
frequency = 0.0
ref_counter = time.perf_counter()
key = ''

while key != 'ok':
    key = remote_control.get_remote_code()
    if key >= '1' and key <= '9':
        if key in ('1', '4', '7'):
            blink_left = True
            blink_right = False
        elif key in ('2', '5', '8'):
            blink_left = True
            blink_right = True
        elif key in ('3', '6', '9'):
            blink_left = False
            blink_right = True
        if key in ('1', '2', '3'):
            frequency = 1.5
        elif key in ('4', '5', '6'):
            frequency = 1.0
        elif key in ('7', '8', '9'):
            frequency = 0.5
    elif key == '0':
        blink_left = False
        blink_right = False
        frequency = 0.0
        robot.blinker_off(0)
        robot.blinker_off(1)     
    elif key == 'ok':
        continue
    
    if frequency > 0.0:
        now_counter = time.perf_counter() 
        cumul_time = now_counter - ref_counter
        if cumul_time > frequency / 2.0:
            ref_counter = now_counter
            if blink_right and current_set_on:
                robot.blinker_on(0)
            else:
                robot.blinker_off(0)
            if blink_left and current_set_on:
                robot.blinker_on(1)
            else:
                robot.blinker_off(1)
            current_set_on = not current_set_on
            
robot.blinker_off(0)
robot.blinker_off(1)

print("... fin de l'opération des phares")

---
### Exercice 3.5. *
Utilisez la télécommande pour contrôler les phares. Ce programme modifie l'état des phares selon une certaine logique. Voici le sens à donner aux touches :
 - Les 3 paramètres à manipuler sont :
  - phare à faire clignoter :
    - `*` : bascule l'état de clignotement du phare de gauche (clignote - ne clignote pas - clignote - ne clignote pas - ...)
    - `#` : bascule l'état de clignotement du phare de droite (clignote - ne clignote pas - clignote - ne clignote pas - ...)
    - par défaut, les phares clignotent
  - durée du cycle :
    - débute à 1.0 secondes
    - `left` : réduit la durée du cycle 
      - incrément par interval de 0.25
      - minimum de 0.25 seconde
    - `right` : augmente la durée du cycle 
      - incrément par interval de 0.25
      - maximum de 2.5 seconde
  - ratio ouvert/fermé :
    - débute à 0.5
    - `up` : augmente la proportion où le phare est allumé 
      - incrément par interval de 0.1
      - maximum de 1.0
    - `down` : réduit la proportion où le phare est allumé 
      - incrément par interval de 0.1
      - minimum de 0.0
 - `0` : arrête tout clignotement et remet toutes les valeurs par défaut
 - `ok` arrête le programme
 
Vous devez vous assurer qu'il y ait une forme de temporisation sur la lecture de la télécommande pour ne pas passer d'un état à l'autre trop rapidement. Par exemple, un délai d'un quart de seconde peut être satisfaisant. Assurez-vous que les phares soient éteints à la sortie du programme.

**Attention** : à cause de la lecture de la télécommande, vous ne pouvez pas faire de `wait` pour bloquer le programme!

In [ ]:
# Solution
now_counter = time.perf_counter() 

blink_left = True
blink_right = True
cycle_duration = 1.0
percent_on = 0.5

cycle_increment = 0.25
percent_increment = 0.1
current_set_on = True
ref_blink_counter = now_counter 

remote_cycle_duration = 0.25
ref_remote_counter = now_counter
keycodes = ['', 'up', 'left', 'ok', 'right', 'down', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '0', '#']
key = ''

count = 0
while key != 'ok':
    now_counter = time.perf_counter() 
    
    cumul_remote_time = now_counter - ref_remote_counter
    if cumul_remote_time > remote_cycle_duration:
        key_code = remote_control.read()
        key = keycodes[key_code]
        
        if key == '':
            ref_remote_counter = 0.0
        else:
            ref_remote_counter = now_counter
            if key == '*':
                blink_left = not blink_left
            elif key == '#':
                blink_right = not blink_right
            elif key == 'left':
                cycle_duration = max(0.25, cycle_duration - cycle_increment)
            elif key == 'right':
                cycle_duration = min(cycle_duration + cycle_increment, 2.5)
            elif key == 'up':
                percent_on = min(percent_on + percent_increment, 1.0)
            elif key == 'down':
                percent_on = max(0.0, percent_on - percent_increment)
            elif key == '0':
                blink_left = False
                blink_right = False
                cycle_duration = 1.0
                percent_on = 0.5
                robot.blinker_off(0)
                robot.blinker_off(1)
            elif key == 'ok':
                continue
    
    cumul_blink_time = now_counter - ref_blink_counter
    if (current_set_on and cumul_blink_time > cycle_duration * (1.0 - percent_on)) or (not current_set_on and cumul_blink_time > cycle_duration * percent_on):
        if blink_right and current_set_on:
            robot.blinker_on(0)
        else:
            robot.blinker_off(0)
        if blink_left and current_set_on:
            robot.blinker_on(1)
        else:
            robot.blinker_off(1)
        ref_blink_counter = now_counter
        current_set_on = not current_set_on
        
    print(f'\r| blink_left : {blink_left} | blink_right : {blink_right} | cycle_duration : {cycle_duration:0.2f} | percent_on : {percent_on * 100:0.0f} % | ', sep='', end=' ' * 100)
            
robot.blinker_off(0)
robot.blinker_off(1)

---
### Exercice 3.6. *
Faites l'implémentation d'une petite librairie réalisant le patron de conception observateur (_observer_) avec la télécommande. Autrement dit, vous implémentez une boucle de lecture en permanance qui, appelle les fonctions enregistrées pour les évènements `on_key_pressed` et `on_key_released`. Dans les deux cas, on doit faire passer en argument la touche concernée.

Lorque votre outil est disponible, utilisez-le pour cette application : simplement afficher une chaîne de caractère qui décrit un évènement `on_key_pressed` ou `on_key_released` lorqu'il arrive.

In [ ]:
# Solution
from abc import ABC, abstractmethod

class RobotRemote:
    class Observer(ABC):
        @abstractmethod
        def update(self, key_pressed : bool, key : str): 
            '''key_pressed == true  => key was pressed
               key_pressed == false => key was released
               key => the key action key'''
            pass
    
    def __init__(self, remote_control):
        self.__remote_control = remote_control
        self.__keycodes = ['', 'up', 'left', 'ok', 'right', 'down', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '0', '#']
        self.__observers = []
        self.__current_key_code = 0
        
    def add_observer(self, observer):
        if not isinstance(observer, RobotRemote.Observer):
            raise ValueError('observer must be an RobotRemote.Observer object')
        self.__observers.append(observer)
        
    def update(self):
        key_code = self.__remote_control.read()
        if key_code != self.__current_key_code:
            if self.__current_key_code != 0:
                for observer in self.__observers:
                    observer.update(False, self.__keycodes[self.__current_key_code])
            self.__current_key_code = key_code
            if self.__current_key_code != 0:
                for observer in self.__observers:
                    observer.update(True, self.__keycodes[self.__current_key_code])

class KeyPrintAndWatchOk(RobotRemote.Observer):
    def __init__(self):
        self.__ok_was_pressed = False
        
    @property
    def ok_was_pressed(self):
        return self.__ok_was_pressed
        
    def update(self, key_pressed : bool, key : str):
        if key == 'ok':
            self.__ok_was_pressed = True
        print(f'\rKey \'{key}\' was {"pressed" if key_pressed else "released" }.', end=' ' * 50)
        

# Début du programme
# Allocation des objets et enregistrement d'un observateur
remote_control = RobotRemote(remote_control)
obs = KeyPrintAndWatchOk()
remote_control.add_observer(obs)


print("Début de l'écoute...")
# boucle tant que la touche ok n'est pas appuyé
while not obs.ok_was_pressed:
    remote_control.update()

print("\n... fin de l'écoute.")